Classification Problem

IMPORING MODULE

In [99]:
import pandas as pd
import numpy as np 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

In [100]:
data = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")
pid = test["Patient Id"]

Removing columns which will not impact out target

In [101]:
data.drop(["Patient Id","Institute Name", "Patient First Name", "Family Name", "Father's name"],axis=1,inplace=True) 

In [102]:
test.drop(["Patient Id","Institute Name", "Patient First Name", "Family Name", "Father's name"],axis=1,inplace=True) 

In [103]:
data["Symptom 3"].isnull()                                                    

0        False
1        False
2        False
3        False
4        False
         ...  
22078    False
22079    False
22080    False
22081    False
22082    False
Name: Symptom 3, Length: 22083, dtype: bool

In [104]:
for i in data.columns:
    t = data[i].isnull()
    for j in range(len(data[i])):
        if t[j]==True: 
            for k in range(len(data[i])): 
                if t[k]==False:
                    data[i][j]=data[i][k]
                    break    

<ipython-input-104-a8cbfdc55cd9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=data[i][k]


In [105]:
for i in test.columns:
    t = test[i].isnull()
    for j in range(len(test[i])):
        if t[j]==True: 
            for k in range(len(test[i])): 
                if t[k]==False:
                    test[i][j]=test[i][k]
                    break

<ipython-input-105-ee19c447c5e8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[i][j]=test[i][k]


In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22083 entries, 0 to 22082
Data columns (total 40 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Age                                       22083 non-null  float64
 1   Genes in mother's side                            22083 non-null  object 
 2   Inherited from father                             22083 non-null  object 
 3   Maternal gene                                     22083 non-null  object 
 4   Paternal gene                                     22083 non-null  object 
 5   Blood cell count (mcL)                            22083 non-null  float64
 6   Mother's age                                      22083 non-null  float64
 7   Father's age                                      22083 non-null  float64
 8   Location of Institute                             22083 non-null  object 
 9   Status           

In [107]:
data['Disorder Subclass'].nunique()   

9

In [108]:
lable_y = { 'Mitochondrial genetic inheritance disorders': 0,
                  'Multifactorial genetic inheritance disorders': 1,
                  'Single-gene inheritance diseases':2
}
lable_z = {"Leber's hereditary optic neuropathy":0,
              'Cystic fibrosis':1,
              'Diabetes':2,
              'Leigh syndrome':3,
              'Cancer':4,
              'Tay-Sachs':5,
              'Hemochromatosis':6,
               'Mitochondrial myopathy':7,
                "Alzheimer's":8
 }

In [109]:
data["Genetic Disorder"] = data["Genetic Disorder"].map(lable_y)
data["Disorder Subclass"] = data["Disorder Subclass"].map(lable_z)

In [110]:
le = LabelEncoder()

one of encoding

In [111]:
for i in data.columns:
    if isinstance(data[i][0],str):
        data[i] = le.fit_transform(data[i])

In [112]:
for i in test.columns:
    if isinstance(test[i][0],str):
        test[i] = le.fit_transform(test[i])

Dividing data into three parts

In [113]:
x = data.drop(["Genetic Disorder","Disorder Subclass"],axis=1)
y = data["Genetic Disorder"]
z = data["Disorder Subclass"]

In [114]:
xtest = test

In [115]:
print(x.shape,xtest.shape)

(22083, 38) (9465, 38)


standard scaling

In [116]:
x = StandardScaler().fit_transform(x)
xtest = StandardScaler().fit_transform(xtest)

random forest

In [117]:
from sklearn.ensemble import RandomForestClassifier
rfs = RandomForestClassifier(n_estimators=500)
rfs.fit(x, y)
rfspy = rfs.predict(xtest)  

for z parameter

In [118]:
from sklearn.ensemble import RandomForestClassifier
lrs = RandomForestClassifier(n_estimators=500)
lrs.fit(x, z)
lrsp = lrs.predict(xtest)

In [119]:
lable_y = { 0:'Mitochondrial genetic inheritance disorders',
                  1:'Multifactorial genetic inheritance disorders',
                 2: 'Single-gene inheritance diseases'
}
lable_z = {0:"Leber's hereditary optic neuropathy",
              1:'Cystic fibrosis',
              2:'Diabetes',
              3:'Leigh syndrome',
              4:'Cancer',
              5:'Tay-Sachs',
              6:'Hemochromatosis',
               7:'Mitochondrial myopathy',
                8:"Alzheimer's"
 }

In [123]:
res = {
    "Patient Id":pid,
    "Genetic Disorder":rfspy,
    "Disorder Subclass":lrsp
}

In [125]:
res = pd.DataFrame(res)

In [126]:
res["Genetic Disorder"] = res["Genetic Disorder"].map(lable_y)
res["Disorder Subclass"] = res["Disorder Subclass"].map(lable_z)

In [129]:
res.to_csv("submission1.csv", index=False)